In [1]:
%run supportvectors-common.ipynb



<center><img src="https://d4x5p7s4.rocketcdn.me/wp-content/uploads/2016/03/logo-poster-smaller.png"/> </center>
<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# LangChain Tools

As we learned in a previous lab, agents in `langchain` create dynamic chains based on the context and user inputs, using the appropriate tools and large language models.

### Learning goal

In this lab, we will develop more fluency with the some of the tools that are available as part of the `langchain` ecosystem. Later on, in a subsequent lab, we will learn to create our own **custom tools**.

Let us begin by installing a few libraries that we will need:

In [2]:
# Note: uncomment this only if these libraries are not already installed.

#!pip install arxiv
#!pip install wikipedia

Then, let us import the components we will need.

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.utilities import ArxivAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, Tool, initialize_agent, tool
from langchain.tools import BaseTool
from langchain.agents import load_tools
from langchain.chains import LLMChain
from IPython.display import Markdown, HTML

## List of useful tools

`langchain` comes with many tools that help with common tasks. To see the full list, visit the website: https://python.langchain.com/en/latest/modules/agents/tools.html

Let us browse through it and become familiar with the rich ecosystem of tools already available. Furthermore, we can easily create our own custom tools with relative ease.

## The search tool
Let us play with the search tool, and see how it works. First we will invoke it directly. Later, we will make it a part of a `langchain`, and use it with an agent.

In [4]:
search = SerpAPIWrapper()
search.run ('Where is SupportVectors AI Lab located?')

'south Fremont'

Now, let us use it with a chain.

In [5]:
tool_names = ['serpapi']
tools = load_tools(tool_names)
llm   = OpenAI(temperature=0.9) 
agent = initialize_agent(
                        tools   = tools,
                        llm     = llm,
                        agent   = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose = True
                        )

In [6]:
query = """
What is the climate like in the city where SupportVectors AI lab is located? 

Describe the weather with a witty and catchy title, in the format.

<blockquote><h4>[Weather Title] </h4>


[Weather]

Then enumerate and describe 5 pleasant hikes in that neighborhood in the today's weather.


<h4> Hikes</h4>

[Hikes]

</blockquote>
"""

result = agent.run (query)
display(Markdown(result))



> Entering new AgentExecutor chain...
 I need to find out the climate of SupportVectors AI lab's city
Action: Search
Action Input: Climate of SupportVectors AI lab's city
Observation: ['Our workshop environment encourages a creative, fun, and comfortable atmosphere. ... a Big-data cluster lab, with GPU-accelerated servers ...']
Thought: SupportVectors AI lab is located in Hong Kong
Action: Search
Action Input: Weather in Hong Kong
Observation: {'type': 'weather_result', 'temperature': '77', 'unit': 'Fahrenheit', 'precipitation': '16%', 'humidity': '93%', 'wind': '7 mph', 'location': 'Hong Kong', 'date': 'Tuesday 2:00 AM', 'weather': 'Mostly cloudy'}
Thought: I now know the final answer
Final Answer:

<blockquote><h4>Scorcher in the Skyline </h4>

The climate in SupportVectors AI lab’s city, Hong Kong, is mostly cloudy with temperatures around 77°F (25°C), 16% precipitation, 93% humidity, and wind speeds of 7 mph.

<h4>Hikes</h4>

1. Victoria Peak – Get breathtaking views of the skyli

<blockquote><h4>Scorcher in the Skyline </h4>

The climate in SupportVectors AI lab’s city, Hong Kong, is mostly cloudy with temperatures around 77°F (25°C), 16% precipitation, 93% humidity, and wind speeds of 7 mph.

<h4>Hikes</h4>

1. Victoria Peak – Get breathtaking views of the skyline and sea while trekking up almost 2 km of winding paths.

2. Lantau Island – Glimpse beautiful temples and monasteries, swaying bamboo forests, and the silver lake Tai O.

3. Mui Wo – Explore the beaches and nature trails around this “Back Garden of Hong Kong.”

4. Sai Kung – Take in the striking natural beauty of acres of untouched coastline.

5. High Junk Peak – Enjoy some of the city’s best views of the nearby islands from the top of this iconic hill.

</blockquote>

### Arxiv

Let us begin by playing with another utility, to search only in the arxiv repository.



<div class="alert-box alert-info" style="padding-top:30px">

<h4> What is Arxiv?</h4>
  
    
> Arxiv is one of the largest repositories of research work. Most of the new developments in our field of AI show up first as a pre-prints, and only much later some of them make their way into research journals and conference proceedings. As such, it has become the *de-facto* watering hole for researchers to go to, in search of new developments.
> 
> We will use the `arxiv` library, and the `langchain.utilities.ArxivAPIWrapper` class to illustrate the use of this tool.
    
    
</div>


We will begin by instantiating a `langchain` api-wrapper around the `arxiv` library. Then we will run it to see it retrieve pertinent pre-prints for us.

We will consider an old paper in high-energy physics that the instructor co-authored:

In [7]:
query = 'hep-ph/9703234'
arxiv = ArxivAPIWrapper()
docs = arxiv.run(query)
print(docs)

Published: 1997-07-14
Title: Hyperfine Splitting of Low-Lying Heavy Baryons
Authors: M. Harada, A. Qamar, F. Sannino, J. Schechter, H. Weigel
Summary: We calculate the next-to-leading order contribution to the masses of the
heavy baryons in the bound state approach for baryons containing a heavy quark.
These $1/N_C$ corrections arise when states of good spin and isospin are
generated from the background soliton of the light meson fields. Our study is
motivated by the previously established result that light vector meson fields
are required for this soliton in order to reasonably describe the spectrum of
both the light and the heavy baryons. We note that the inclusion of light
vector mesons significantly improves the agreement of the predicted hyperfine
splitting with experiment. A number of aspects of this somewhat complicated
calculation are discussed in detail.


Let us now illustrate its use.

In [9]:
from pydantic import BaseModel, Field

class ArxivInput(BaseModel):
    preprint: str = Field ()

tool = Tool (
            name         = 'arxiv',
            func         = arxiv.run,
            args_schema  = ArxivInput,
            description  = 'Search the arxiv pre-print repository')

ValidationError: 1 validation error for Tool
args_schema
  subclass of BaseModel expected (type=type_error.subclass; expected_class=BaseModel)

In [11]:
tools = [tool]
llm   = OpenAI(temperature = 0.0)
agent = initialize_agent(
                        tools   = tools,
                        llm     = llm,
                        agent   = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose = True
                         )

AttributeError: 'function' object has no attribute 'is_single_input'

In [ ]:
query = 'Search for papers dealing with "Attention is all you need". Show the first 5 results.'
results = agent.run(query)
print (results)

### Homework

Convert the bash command tool below to a custom tool (even if one exists for it in the `langchain` ecosystem), and run it to find all files whose names contain the sub-string 'langchain'.

We show the basics of using the utility below.

In [ ]:
from langchain.utilities import BashProcess
bash = BashProcess()
print (bash.run('ls -l'))

## What we have learned?

In this lab, we learned about:

* we have learned about the ecosystem of available tools
* how we can use the tools to good effect in creating useful language chains
* became familiar with `arxiv` and `wikipedia` libraries, and used them as tools
